In [5]:
import sympy as sp
from itertools import combinations
import numpy as np
#import json
import re

In [34]:
class CliffordNumber():
    def __init__(self, spacetime_dim, matrix_symbol = "g", component_symbol = "x", metric = None):
        self.spacetime_dim = spacetime_dim
        self.clifford_dim = np.power(2, self.spacetime_dim)
#         print("The SpaceTime dim  is {}".format(self.spacetime_dim))
#         print("The Clifford dim  is {}".format(self.clifford_dim))
        
        # We are going to build a set of Clifford basis using the following symbol
        self.mysymbol = matrix_symbol
        self.mysympy_base = component_symbol
        self.clifford_map = self.set_mapping(spacetime_dim, self.mysymbol, self.mysympy_base )
               
        #self.show_clifford_variables()
        
    def add_single_multivectors(self, mv, scalar):
        try:
            self.clifford_map[mv]['scalar'] = self.clifford_map[mv]['scalar'] + scalar
            #print("Added {} with scalar = {}".format(mv, scalar))
        except Exception:
            new_mv, sign = self.order_mv(mv)
            self.clifford_map[new_mv]['scalar'] = self.clifford_map[new_mv]['scalar'] + sign*scalar
            #print("Added {} with scalar = {} after re-order".format(new_mv, sign*scalar))
            
    def add_symbol_multivectors(self, mv, symbol):
        try:
            self.clifford_map[mv]['sympy'] = self.clifford_map[mv]['sympy'] + symbol
            #print("Added {} with component = {}".format(mv, symbol))
        except Exception:
            new_mv, sign = self.order_mv(mv)
            self.clifford_map[new_mv]['sympy'] = self.clifford_map[new_mv]['sympy'] + sign * symbol
            #print("Added {} with component = {} after re-order".format(new_mv, sign*symbol))
        
    def show_clifford_variables(self):
        print("\nClifford Components = ")
        for key, value in self.clifford_map.items():
            if sp.simplify(value['sympy']) == 0:# It doesn't work
                pass
            else:
                print("({}){}".format(value['sympy'], key))
        print("\n")
            
    def show_clifford_values(self):
        print("\nClifford values = ")
        for key, value in self.clifford_map.items():
            s = value['scalar']
            if s !=0:
                print("{}{}".format(s, key))
        
    def set_mapping(self, spacetime_dim, mysymbol, mysympy_base):
        elements = [i for i in range(spacetime_dim)]
        mymap = {}
        
               
        for rank in range(spacetime_dim+1):
                    
            # Calculate all combinations inside a rank and move them to list
            all_combin_in_rank= list(combinations(elements, rank))
            
            # Going along all the combin for a certain rank
            index_in_rank = 0
            for single_combin in all_combin_in_rank:
                #print(single_combin)
                
                # Here all the info we want to associate to key value
                myinfo = {} 
                myinfo['rank'] = rank
                myinfo['index_in_rank'] = index_in_rank
                myinfo['combin_in_rank'] = single_combin
                myinfo['scalar'] = 0
                #myinfo['sympy'] = ... sp.Symbol('s')
                
                # The following elements should be built according to the indeces
                mykey = ""
                mysympy = mysympy_base
                
                if rank == 0:# First object is special
                    if mysympy_base == "":
                        myinfo['sympy'] = sp.Symbol("")
                    else:
                        #myinfo['sympy'] = sp.Symbol('{}s'.format(mysympy_base)) # without saving the proper dynamic attribute
                        attribute_name = '{}s'.format(mysympy_base)
                        setattr(self, attribute_name, sp.Symbol('{}s'.format(mysympy_base)))
                        myinfo['sympy'] = getattr(self, attribute_name)
                        
                        
                    mymap["[I]"] = myinfo
                else:
                    for r in range(rank):
                        mysympy = mysympy + "{}".format(single_combin[r])
                        mykey = mykey + "[{}{}]".format(mysymbol, single_combin[r])
                    if mysympy_base == "":
                        myinfo['sympy'] = sp.Symbol("")
                    else:    
                        #myinfo['sympy'] = sp.Symbol(mysympy) # without saving the proper dynamic attrib
                        attribute_name = mysympy
                        setattr(self, attribute_name, sp.Symbol(mysympy))
                        myinfo['sympy'] = getattr(self, attribute_name)
                        
                        
                    mymap[mykey] = myinfo
                    
                
                # Here we increase the index
                index_in_rank = index_in_rank + 1
            
#         formatted_json =json.dumps(mymap,indent=4)
#         print(formatted_json)
        #print(mymap)
        return mymap

    def get_module_by_index(self, index):
        if index==0:
            return 1
        else:
            return -1

    def order_mv(self, mv):
        #print("{} to be ordered".format(mv))
        str_mv = re.split("\]\[{}|\[{}|\]".format(self.mysymbol, self.mysymbol), mv)[1:-1]
        int_mv = [int(j) for j in str_mv]
        
        sign = +1
        variable_length = len(int_mv)
        if variable_length >=2:
            num_of_changes = 1
            while (num_of_changes>0):
                num_of_changes = 0 # reset changes for a new search
                i = 0
                while i < (variable_length-1):
                    #print("FB1 {}".format(i))
                    comp_a = int_mv[i] # First comparator
                    comp_b = int_mv[i+1] # Second comparator
                    
                    if comp_a > comp_b: # If greater, we interchange
                        num_of_changes = num_of_changes + 1 # increase changes
                        sign = sign*(-1) # sign inverted
                        # Interchange procedure
                        int_mv[i] = comp_b
                        int_mv[i+1] = comp_a
                        #print("{} after interchange".format(int_mv))
                    elif comp_a == comp_b: # if equals, we delete
                        num_of_changes = num_of_changes + 1 # increase changes
                        sign = sign*self.get_module_by_index(int_mv[i]) # sign updated according to the metric
                        #print("indeces = {},{} should be deleted".format(i, i+1))
                        # Deletion procedure
                        ir = i+2
                        int_mv = int_mv[:i]+ int_mv[ir:] # delete two items
                        #print("{} after deletion".format(int_mv))
                        # Once deleted we should update the lenght and check that we are still in...
                        variable_length=variable_length-2
                        if i < variable_length - 3:
                            i = i-1
                            #print("step one index in this sequence")
                        else:
                            #print("no more checks in this sequence")
                            break
                    i = i+1     
                    #print("FB2 i={} num_of_changes={}".format(i, num_of_changes))
                    
            # Rebuild the mv
            l = len(int_mv)
            new_mv = ""
            for i in range(l):
                new_mv = new_mv + "[{}{}]".format(self.mysymbol, int_mv[i])
                
            if new_mv == "":
                new_mv = "[I]"
            return new_mv, sign      
  
    def __add__(self, other):
        print("############### Clifford Sum ############### ")
        if isinstance(other, CliffordNumber):
            output_cn = CliffordNumber(spacetime_dim = self.spacetime_dim, matrix_symbol = "g", component_symbol = "")
            
            for key, value in self.clifford_map.items():
                other_value = other.clifford_map[key]
                sum_value = value['sympy'] + other_value['sympy']
                output_cn.add_symbol_multivectors(key, sum_value)
            return output_cn
        else:
            raise TypeError("Unsupported operand type for +")
        
    def __sub__(self, other):
        print("############### Clifford Substraction ############### ")
        if isinstance(other, CliffordNumber):
            output_cn = CliffordNumber(spacetime_dim = self.spacetime_dim, matrix_symbol = "g", component_symbol = "")
            
            for key, value in self.clifford_map.items():
                other_value = other.clifford_map[key]
                sum_value = value['sympy'] - other_value['sympy']
                output_cn.add_symbol_multivectors(key, sum_value)
            return output_cn
        else:
            raise TypeError("Unsupported operand type for -")
        
    def __truediv__(self, other):
        print("############### Clifford Division ############### ")
        if isinstance(other, (int,float)):
            if other == 0:
                raise ValueError("Division by zero is not allowed.")
            
            output_cn = CliffordNumber(spacetime_dim = self.spacetime_dim, matrix_symbol = "g", component_symbol = "")
            
            for key, value in self.clifford_map.items():
                sum_value = value['sympy'] / other
                output_cn.add_symbol_multivectors(key, sum_value)
            return output_cn
        else:
            raise TypeError("Unsupported operand type for /")
        
    def __mul__(self, other):
        print("############### Clifford Product ############### ")
        if isinstance(other, CliffordNumber):
            output_cn = CliffordNumber(spacetime_dim = self.spacetime_dim, matrix_symbol = "g", component_symbol = "")
            for key1, value1 in self.clifford_map.items():
                for key2, value2 in other.clifford_map.items():
                    #print("{}{}{}{}".format(value1['sympy'], key1,value2['sympy'], key2 ))
                    product_value = value1['sympy'] * value2['sympy']

                    # If one of the two elemets is the unit-vector
                    if key1 == "[I]":
                        key1 = ""
                    if key2 == "[I]":
                        key2 = ""

                    product_key = key1 + key2
                    
                    # If oboth elemets are the unit-vector
                    if product_key == "":
                        product_key = "[I]"

                    output_cn.add_symbol_multivectors(product_key, product_value)
            return output_cn
        else:
            raise TypeError("Unsupported operand type for *")

In [37]:
cn = CliffordNumber(spacetime_dim=4, matrix_symbol = "g", component_symbol = "x")

cn.add_single_multivectors('[g0][g0][g2]', 3)
cn.add_single_multivectors('[g1][g3][g0][g2]', 7)
cn.add_single_multivectors('[g2][g1]', 7)
cn.show_clifford_values()

# Define two generic Clifford numbers:
dim = 2
cnn1 = CliffordNumber(spacetime_dim=dim, matrix_symbol = "g", component_symbol = "x")
cnn2 = CliffordNumber(spacetime_dim=dim, matrix_symbol = "g", component_symbol = "y")

# test symbolic sum of two Cliffor numbers into a new one:
cnn_sum = cnn1 + cnn2
cnn_sum.show_clifford_variables()

# test symbolic product of two Cliffor numbers into a new one:
cnn_product = cnn1 * cnn2
cnn_product.show_clifford_variables()

# Test cross product
cnn_product_inverted = cnn2 * cnn1
cnn_cross_product = (cnn_product - cnn_product_inverted)/2
cnn_cross_product.show_clifford_variables()

#Test dot product
cnn_dot_product = (cnn_product + cnn_product_inverted)/2
cnn_dot_product.show_clifford_variables()


#*There is a bug with 'empty' SymPy expressions
# We observe some extra terms in the outputs.
# To be fixed.


Clifford values = 
3[g2]
-7[g1][g2]
-7[g0][g1][g2][g3]
############### Clifford Sum ############### 

Clifford Components = 
( + xs + ys)[I]
( + x0 + y0)[g0]
( + x1 + y1)[g1]
( + x01 + y01)[g0][g1]


############### Clifford Product ############### 

Clifford Components = 
( + x0*y0 + x01*y01 - x1*y1 + xs*ys)[I]
( + x0*ys - x01*y1 + x1*y01 + xs*y0)[g0]
( + x0*y01 - x01*y0 + x1*ys + xs*y1)[g1]
( + x0*y1 + x01*ys - x1*y0 + xs*y01)[g0][g1]


############### Clifford Product ############### 
############### Clifford Substraction ############### 
############### Clifford Division ############### 

Clifford Components = 
(3*/2)[I]
(3*/2 - x01*y1 + x1*y01)[g0]
(3*/2 + x0*y01 - x01*y0)[g1]
(3*/2 + x0*y1 - x1*y0)[g0][g1]


############### Clifford Sum ############### 
############### Clifford Division ############### 

Clifford Components = 
(5*/2 + x0*y0 + x01*y01 - x1*y1 + xs*ys)[I]
(5*/2 + x0*ys + xs*y0)[g0]
(5*/2 + x1*ys + xs*y1)[g1]
(5*/2 + x01*ys + xs*y01)[g0][g1]




In [41]:
# SymPy object of the product MV, component of [g0]
sympy_g0 = cnn_product.clifford_map["[g0]"]['sympy']
print(sympy_g0)

# SymPy object of the input1, component of [g1] => x1
sysympy_product = cnn_product.clifford_map["[g1]"]['sympy']
print(sysympy_product)

# We evaluate xo in the previous SymPy object
sympy_g0.subs(sysympy_product, 5)

 + x0*ys - x01*y1 + x1*y01 + xs*y0
 + x0*y01 - x01*y0 + x1*ys + xs*y1


 + x0*ys - x01*y1 + x1*y01 + xs*y0

In [42]:
# SymPy object of the product, component of [I]
sympy_I = cnn_cf.clifford_map["[I]"]['sympy']
print(sympy_I)
# We evaluate x01 = 2 y y01 = 3
substitutions = {cnn1.x01: 2, cnn2.y01: 3}
print(sympy_I.subs(substitutions ))

 + x0*y0 + x01*y01 - x1*y1 + xs*ys
 + x0*y0 - x1*y1 + xs*ys + 6
